In [1]:
import os
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [2]:
booksdata_filename = '/FileStore/tables/oqkeyakf1491168586469/BX_Book_Ratings-05a61.csv'
boooksdata_df_schema = StructType([StructField('User', IntegerType()), StructField('ISBN', IntegerType()), StructField('Rating', IntegerType())]) 

booksdata_df = sqlContext.read.format('com.databricks.spark.csv').options(delimiter = ',', header=True,inferSchema=False).schema(boooksdata_df_schema).load(booksdata_filename)

In [3]:
seed = 1800009193L
(split_60_df, split_a_20_df, split_b_20_df) = booksdata_df.randomSplit([0.6, 0.2, 0.2], seed = seed)
training_df = split_60_df.cache()
validation_df = split_a_20_df.cache()
test_df = split_b_20_df.cache()

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
als = ALS()
als.setMaxIter(5).setSeed(seed).setItemCol("ISBN").setRatingCol("Rating").setUserCol("User")
#reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Rating", metricName="rmse")
als.setParams(rank = 4, regParam = 0.15)

In [5]:
als.fit(training_df)

In [6]:
training_df